
## Table of  Contents
1. [Initial Coverage Rate](#1)
1. [Coverage Rate after Data Cleaning](#2)
1. [Handelling Missing Values](#3)
1. [Correcting Misspellings](#4)

In [3]:
!pip install geotext
!pip install pycountry
!pip install fuzzywuzzy
!pip install us
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=eb882940bf85e4c9c636843a07a70bb97887eace6a6938d428bc73cf3f1462c6
  Stored in directory: /root/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.9 MB/s eta 0:00:00
  Created wheel for us: filename=us-3.1.1-py3-none-any.whl size=12547 sha256=e11226ea83e54880e72c660f3532a363022cdcd1a1a323d2b65687f10496b707
  Stored in directory: /root/.cache/pip/wheels/da/f9/64/9669135a7ce533faf78e3b32259c505f964f03b069be6844f8
Successfully built us
     ━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geotext import GeoText
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pycountry
import re
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from termcolor import colored
import nltk
from nltk.corpus import words
import us
from spellchecker import SpellChecker
import difflib
from IPython.display import display

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls /content/drive/MyDrive/NLP_address_correction/

all_addresses.csv    approach2Result.csv	 test.csv
approach1Result.csv  matching_rows_modified.csv  test_geo_data2.csv


In [7]:
# Reading test.csv and all_addresses.csv
test_df = pd.read_csv('/content/drive/MyDrive/NLP_address_correction/test.csv')
all_addresses_df = pd.read_csv('/content/drive/MyDrive/NLP_address_correction/all_addresses.csv')

In [8]:
def display_dataframe(dataframe, dataframe_name):
    print(dataframe_name)
    display(dataframe)
    print("\n" * 2)  # Add two blank lines between the tables

# Replace these lines with the names of your DataFrames and the names you want to display
display_dataframe(test_df, 'test_df')
display_dataframe(all_addresses_df, 'all_addresses_df')


test_df


,uid,address,city,state,zip
0,1,941 Thorpe St,Rock Springs,WY,82901.0
1,2,2809 HARRIS DR,Antioch,CA,94509.0
2,3,1441 Eastlake Ave,Los Angeles,CA,90033.0
3,4,7 ucayptus,Newport Beach,CA,92657.0
4,5,1400 Lachman Ln,NaN,CA,90272.0
...,...,...,...,...,...
99244,99996,2020 s moreno dr,apache junction,az,85120.0
99245,99997,10641 Missouri Ave,Los Angeles,CA,90025.0
99246,99998,27905 Tyler Ln,Santa Clarita,CA,91387.0
99247,99999,6259 W 55th St,Chicago,NaN,60638.0





all_addresses_df


,address,city,state,zip
0,941 Thorpe St,Rock Springs,WY,82901
1,2809 HARRIS DR,Antioch,CA,94509
2,1441 Eastlake Ave,Los Angeles,CA,90033
3,7 Eucalyptus,Newport Beach,CA,92657
4,1400 Lachman Ln,Los Angeles,CA,90272
...,...,...,...,...
129995,4477 Camrose Ave,San Diego,CA,92122
129996,13 STONEWALL,Irvine,CA,92620
129997,3456 Nouveau Way,Gold River,CA,95670
129998,9115 Bungalow Way,Elk Grove,CA,95758


In [9]:
# Display data types of each column
test_df_types = test_df.dtypes
# Count missing values in each column
test_df_missing_values = test_df.isna().sum()

# Create a new DataFrame to combine data types and missing values
summary_df = pd.DataFrame({
    'Columns': test_df.columns,
    'Data Type': test_df_types,
    'Missing Values': test_df_missing_values
})

# Display the combined summary DataFrame
print("test data Summary:")
# Reset index and drop the old index column
summary_df = summary_df.reset_index(drop=True)
summary_df

test data Summary:


,Columns,Data Type,Missing Values
0,uid,int64,0
1,address,object,0
2,city,object,1282
3,state,object,7433
4,zip,float64,7555


### <a id='1'>1. Initial Coverage Rate</a>  

# Add 'uid' coloumn (primary key) to all_addresses_df.
#### - A new column 'uid' is added to the all_addresses_df DataFrame. This column is assigned unique identifiers based on the DataFrame's index.
#### - The 'uid' column is essential for matching rows between datasets and tracking improvements in coverage.

In [10]:
all_addresses_df['uid'] = all_addresses_df.index+1

In [11]:
# In this section, the same preprocessing steps to both the "test.csv" and "all_addresses.csv" datasets are applied, ensuring uniform formatting throughout.

# Converte 'zip'into int.
test_df['zip'] = test_df['zip'].fillna(0).astype(int)
all_addresses_df.zip =pd.to_numeric(all_addresses_df.zip, errors='coerce').fillna(0).astype('int')

# Drop duplicated rows and rows with '#NAME?|#NAM?'
# test_df.drop_duplicates(inplace=True)
# all_addresses_df.drop_duplicates(inplace=True)
test_df = test_df[~test_df['address'].str.contains('#NAME?|#NAM?', na=False)]
all_addresses_df = all_addresses_df[all_addresses_df['address'] != '#NAME?|#NAM?']

# Convert text into lowercase
test_df.loc[:, 'address'] = test_df['address'].str.lower()
test_df.loc[:, 'city'] = test_df['city'].str.lower()
test_df.loc[:, 'state'] = test_df['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()


# Finding same rows in two files
correct_addresses_df = pd.merge(test_df, all_addresses_df, on=['uid','address', 'city', 'state', 'zip'], how='inner')

# Calculation of initial coverage rate
initial_coverage_rate = len(correct_addresses_df)/len(test_df)*100

<ipython-input-11-4b83d685855b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'address'] = test_df['address'].str.lower()


In [12]:
print('Length of test data: ', len(test_df))
print('Length of test addresses: ', len(all_addresses_df))
print('Initial number of correct addresses: ', len(correct_addresses_df))
print('Initial coverage rate: ', initial_coverage_rate)

Length of test data:  99017
Length of test addresses:  130000
Initial number of correct addresses:  62769
Initial coverage rate:  63.39214478321904


## <a id='2'>2. Coverage Rate after Data Cleaning</a>  
Data *Cleaning*

In [13]:
# 131439th Ave -> 1314 39th Ave
def correct_address(address):
    parts = address.split()
    first_part = parts[0]
    rest_part = ' '.join(parts[1:])

    match = re.match(r'^(\d{4})(\d+)(t|th|n|nd)$', first_part)
    if match:
        first_digits = match.group(1)
        rest_digits = match.group(2)
        ending = match.group(3)

        if ending == 't':
            ending = 'th'
        elif ending == 'n':
            ending = 'nd'

        corrected_first_part = f"{first_digits} {rest_digits}{ending}"
        corrected_address = f"{corrected_first_part} {rest_part}"
        return corrected_address
    else:
        return address


# Function to add spaces between capitalized words and abbreviations
def add_spaces_capWords_abbrs(address):
    abbreviations = ["Terrace", "Way", "Walk", "St", "Rd", "Ave", "Cl", "Ct",
                 "Cres","Cir", "Blvd", "Dr", "Ln", "Pl", "Sq", "Pde", "Cct"]

    pattern = re.compile(r'(?<=[a-z])([A-Z][a-z]+)(?<=\b)')
    def repl(match):
        word = match.group(0)
        if word in abbreviations and not address.startswith(word):
            return f' {word}'
        return word

    modified_address = pattern.sub(repl, address)
    return modified_address


# Function to modify address if the condition is met
def modify_UpperCase_end_ST(address):
    address = address['address']
    if len(address.split()) == 1 and address.split()[0][-2:].lower() == 'st':
        return address[0:-2] + " " + address[-2:]
    return address

def state_name_to_abbr(state_name):
    if isinstance(state_name, str) and state_name.strip() != "":
        try:
            state = us.states.lookup(state_name)
            return state.abbr
        except AttributeError:
            return None
    return None

def test_data_preprocess(df):
    abbreviations = ["Terrace", "Way", "Walk", "St", "Rd", "Ave", "Cl", "Ct",
                 "Cres","Cir", "Blvd", "Dr", "Ln", "Pl", "Sq", "Pde", "Cct"]

    # Remove text after comma in 'address' column
    df['address'] = df['address'].str.split(',').str[0]
    # Remove the specified substring
    df['address'] = df['address'].str.replace(r'\s*#\d+', '', regex=True)
    # 131439th Ave -> 1314 39th Ave
    df['address'] = df['address'].apply(correct_address)
    # Separates the digits and letters in the beginning of the address with a space.
    df['address'] = df['address'].str.replace(r'^(\d+)([A-Za-z])', r'\1 \2', regex=True)
    # e.g., "123W456 -> 123 W 456"
    df['address'] = df['address'].str.replace(r'(\d+)\s?([WE])(\d+)', r'\1 \2 \3', regex=True)
    # e.g., "VanSt -> Van St"
    df['address'] = df['address'].apply(add_spaces_capWords_abbrs)
    # e.g., "VANSt -> VAN St"
    df['address'] = df.apply(modify_UpperCase_end_ST, axis=1)
    # Separate capitalized words, except when the first word is 'Mc'
    df['address'] = df['address'].str.replace(r'(?<=[a-z])(?<!Mc)(?<!Mac)([A-Z][a-z]+)', r' \1', regex=True)
    # "WBennett St" -> "W Bennett St" , "1207 SWBennett St" ->"1207 SW Bennett St"
    df['address'] = df['address'].str.replace(r'([A-Z])([A-Z])([A-Z])([a-z]+)', r'\1\2 \3\4', regex=True)
    df['address'] = df['address'].str.replace(r'([A-Z])([A-Z])([a-z]+)', r'\1 \2\3', regex=True)
    # 10h -> 10th
    df['address'] = df['address'].str.replace(r'(\d+)h', r'\1th', regex=True)
    df['address'] = df['address'].str.replace(r'(\d+)d', r'\1nd', regex=True)
    # e.g., "Ave345 -> Ave 345"
    df['address'] = df['address'].str.replace(r"(" + "|".join(abbreviations) + r")(\d+)", r'\1 \2', regex=True)
    # Convert state names to abbreviations
    df['state'] = df['state'].apply(state_name_to_abbr)

    # Remove leading and trailing whitespace from text columns
    df.loc[:, 'address'] = df['address'].str.strip()
    df.loc[:, 'city'] = df['city'].str.strip()
    df.loc[:, 'state'] = df['state'].str.strip()

def Reference_data_preprocess(df):
    # Remove text after comma in 'address' column
    all_addresses_df['address'] = all_addresses_df['address'].str.split(',').str[0]
    # Remove the specified substring
    all_addresses_df['address'] = all_addresses_df['address'].str.replace(r'\s*#\d+', '', regex=True)
    # Remove leading and trailing whitespace from text columns
    all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.strip()
    all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.strip()
    all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.strip()



In [14]:
# Call the preprocessing function on the 'test' and 'all_addresses' DataFrames
test_data_preprocess(test_df)
Reference_data_preprocess(all_addresses_df)

In [15]:
# Convert all to lower()
test_df.loc[:, 'address'] = test_df['address'].str.lower()
test_df.loc[:, 'city'] = test_df['city'].str.lower()
test_df.loc[:, 'state'] = test_df['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()

# Find number of correct addresses by merging datasets on ['uid','address', 'city', 'state', 'zip']
correct_addresses_df = pd.merge(test_df, all_addresses_df, on=['uid','address', 'city', 'state', 'zip'], how='inner')

# Coverage rate
imp_coverage_rate = len(correct_addresses_df)/len(test_df)*100

# Improvement Rate
improvement_rate = ((imp_coverage_rate - initial_coverage_rate) / initial_coverage_rate) * 100
print('Records of test: ', len(test_df))
print('Records of all_addresses: ', len(all_addresses_df))
print('number of correct addresses: ', len(correct_addresses_df))
print('coverage rate: ', imp_coverage_rate)
print("Improvement Rate:", improvement_rate, "%")

Records of test:  99017
Records of all_addresses:  130000
number of correct addresses:  74545
coverage rate:  75.28505206176716
Improvement Rate: 18.760853287450804 %


## <a id='3'>3. Handelling Missing Values</a>  
- Objective: Fill missing values in the dataset for 'city', 'state', and 'zip' using geocoding techniques and external libraries.
- Libraries Used:
  - geopy.geocoders.Nominatim: Used to geocode addresses and extract location information.
  - pycountry: Utilized for converting country names to ISO codes.
  - geotext.GeoText: Employed for extracting city names from raw addresses.
- Benefits:
  - Improved Accuracy: Geocoding provides accurate city, state, and zip information based on the correct street number and street name.
  - Efficient Automation: The process automates the extraction and filling of missing values.
  - Enhanced Data Quality: Filling missing data enhances the reliability and usability of the dataset.


In [16]:
# Replace zeros with NaN in the 'zip' column
test_df['zip'] = test_df['zip'].replace(0, np.nan)
# Replace missing values in the entire DataFrame with empty strings
test_df.fillna(" ", inplace=True)
# Modify the 'zip' column in place to convert '8562.0' to '8562'
test_df['zip'] = test_df['zip'].astype(str).str.replace('.0', '', regex=False)

##### We just find missing values for test_df['address'] coloumn that match with all_addresses_df['address'] column since the Geocoding is not able to provide accurate 'city', 'state', and 'zip' when the 'address' column (Street number and street name) is wrong.



In [17]:
# Convert all to lower()
test_df.loc[:, 'address'] = test_df['address'].str.lower()
test_df.loc[:, 'city'] = test_df['city'].str.lower()
test_df.loc[:, 'state'] = test_df['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()

# Find number of correct addresses by merging datasets on ['uid','address', 'city', 'state', 'zip']
correct_addresses_address_col = pd.merge(test_df, all_addresses_df, on=['uid','address'], how='inner')

# Coverage rate
coverage_rate_address_col = len(correct_addresses_address_col)/len(test_df)*100


print('Records of test: ', len(test_df))
print('Records of all_addresses: ', len(all_addresses_df))
print('number of correct addresses: ', len(correct_addresses_address_col))
print('coverage rate only for address column: ', coverage_rate_address_col)

# Select the desired columns using .loc indexer
selected_columns = correct_addresses_address_col.loc[:, ['uid', 'address', 'city_x', 'state_x', 'zip_x']]
# Rename the columns
selected_columns.rename(columns={'city_x': 'city', 'state_x': 'state', 'zip_x': 'zip'}, inplace=True)
# selected_columns.head(20)


Records of test:  99017
Records of all_addresses:  130000
number of correct addresses:  90757
coverage rate only for address column:  91.65799812153469


In [49]:
# # Create a geocoder instance
# geolocator = Nominatim(user_agent="my_geocoder")
# test_df_modified = selected_columns

# # Loop through rows of test_df_modified
# for index, row in test_df_modified.iterrows():
#   if row['address'] != ' ':
#     # print(row)
#     if row['city'] == ' ' or row['state'] == ' ' or row['zip'] == ' ':

#       city = row['city']
#       state = row['state']
#       zip_code = row['zip']

#       # Construct the full address string
#       full_address = f"{row['address']}, {city}, {state} {zip_code}"
#       print(full_address)
#       # Geocode the address using Nominatim
#       location = geolocator.geocode(full_address, timeout=180)
#       print(location)
#       if location and len(location.address.split(',')) > 4:
#         raw_address = location.raw.get('display_name', '').split(', ')
#         address = raw_address[0]
#         places = GeoText(", ".join(raw_address[1:]))

#         if row['city'] == ' ':

#           if 'county' in (location.address.split(',')[-5].lower()).strip():
#               city = location.address.split(',')[-6]
#           else:
#               city = location.address.split(',')[-5]
#           test_df_modified.at[index, 'city'] = city.strip()
#           print(city)

#         if row['state'] == ' ':
#           state = location.address.split(',')[-3]
#           abbreviated_state = state_name_to_abbr(state)
#           if abbreviated_state:
#               test_df_modified.at[index, 'state'] = state_name_to_abbr(state).strip()
#               print(state)

#         if row['zip'] == ' ':
#           zip_code = location.address.split(',')[-2]
#           test_df_modified.at[index, 'zip'] = zip_code.strip()
#           print(zip_code)


# ## Save the test_df_modified
# # test_df_modified.to_csv('test_geo_data.csv', index=False)

In [18]:
# Load obtained geo data
test_geo_data_file = "/content/drive/MyDrive/NLP_address_correction/test_geo_data2.csv"
test_geo_df = pd.read_csv(test_geo_data_file)
test_geo_df

,uid,address,city,state,zip
0,1,941 thorpe st,rock springs,wy,82901
1,2,2809 harris dr,antioch,ca,94509
2,3,1441 eastlake ave,los angeles,ca,90033
3,5,1400 lachman ln,Los Angeles,ca,90272
4,7,1966 clinton ave,calexico,ca,92231
...,...,...,...,...,...
92821,99996,2020 s moreno dr,apache junction,az,85120
92822,99997,10641 missouri ave,los angeles,ca,90025
92823,99998,27905 tyler ln,santa clarita,ca,91387
92824,99999,6259 w 55th st,chicago,IL,60638


In [19]:
# Perform a left join to match rows based on 'uid'
merged_df = test_df.merge(test_geo_df, on='uid', how='left', suffixes=('', '_geo'))

# Replace missing values in test_df with values from test_geo_df
merged_df['address'] = merged_df['address_geo'].fillna(merged_df['address'])
merged_df['city'] = merged_df['city_geo'].fillna(merged_df['city'])
merged_df['state'] = merged_df['state_geo'].fillna(merged_df['state'])
merged_df['zip'] = merged_df['zip_geo'].fillna(merged_df['zip'])

# Drop the columns from test_geo_df
merged_df.drop(['address_geo', 'city_geo', 'state_geo', 'zip_geo'], axis=1, inplace=True)

# The merged_df now contains updated rows from test_geo_df
# Ensure the size of merged_df is the same as test_df
merged_df = merged_df.iloc[:len(test_df)]

# Print the updated DataFrame
print(merged_df)

         uid                 address                   city state    zip
0          1           941 thorpe st           rock springs    wy  82901
1          2          2809 harris dr                antioch    ca  94509
2          3       1441 eastlake ave            los angeles    ca  90033
3          4              7 ucayptus          newport beach    ca  92657
4          5         1400 lachman ln            Los Angeles    ca  90272
...      ...                     ...                    ...   ...    ...
99012  99269       2336 mountain ave  la crescenta-montrose    ca  91214
99013  99270         7707 mohegan ct                redding    ca  96002
99014  99271           1593 canby rd                redding    ca  96002
99015  99272  1327 mount olympia cir       south lake tahoe    ca       
99016  99273         1210 w 228th st               torrance    ca  90502

[99017 rows x 5 columns]


In [20]:
# Count the number of empty strings in each column
empty_string_counts_before_geo = test_df.applymap(lambda x: x == " ").sum()
empty_string_counts_after_geo = merged_df.applymap(lambda x: x == " ").sum()
# Print the counts
print('number of missing values before handeling missing values: ')
print(empty_string_counts_before_geo)
print('number of missing values after handeling missing values: ')
print(empty_string_counts_after_geo)

number of missing values before handeling missing values: 
uid           0
address       0
city       1278
state      7412
zip        7534
dtype: int64
number of missing values after handeling missing values: 
uid           0
address       0
city         48
state      1291
zip        1174
dtype: int64


In [21]:
df = merged_df
# List of columns to replace empty strings with NaN
columns_to_replace = df.columns.difference(['zip'])
# Replace empty strings with NaN
df[columns_to_replace] = df[columns_to_replace].replace(' ', np.nan)

# Replace empty strings with 0 in the 'zip' column
df.loc[df['zip'] == ' ', 'zip'] = 0
# Converte 'zip'into int.
df.loc[:, 'zip'] = pd.to_numeric(df['zip'], errors='coerce')

<ipython-input-21-e04398026a95>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'zip'] = pd.to_numeric(df['zip'], errors='coerce')


In [22]:
df

,uid,address,city,state,zip
0,1,941 thorpe st,rock springs,wy,82901.0
1,2,2809 harris dr,antioch,ca,94509.0
2,3,1441 eastlake ave,los angeles,ca,90033.0
3,4,7 ucayptus,newport beach,ca,92657.0
4,5,1400 lachman ln,Los Angeles,ca,90272.0
...,...,...,...,...,...
99012,99269,2336 mountain ave,la crescenta-montrose,ca,91214.0
99013,99270,7707 mohegan ct,redding,ca,96002.0
99014,99271,1593 canby rd,redding,ca,96002.0
99015,99272,1327 mount olympia cir,south lake tahoe,ca,0.0


In [23]:
# Convert all to lower()
df.loc[:, 'address'] = df['address'].str.lower()
df.loc[:, 'city'] = df['city'].str.lower()
df.loc[:, 'state'] = df['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()

# Find number of correct addresses by merging datasets on ['uid','address', 'city', 'state', 'zip']
correct_addresses_df = pd.merge(df, all_addresses_df, on=['uid','address', 'city', 'state', 'zip'], how='inner')

# Coverage rate
imp_coverage_rate = len(correct_addresses_df)/len(df)*100

# Improvement Rate
improvement_rate = ((imp_coverage_rate - initial_coverage_rate) / initial_coverage_rate) * 100
print('Records of test: ', len(df))
print('Records of all_addresses: ', len(all_addresses_df))
print('Initial number of correct addresses: ', len(correct_addresses_df))
print('Initial coverage rate: ', imp_coverage_rate)
print("Improvement Rate:", improvement_rate, "%")


Records of test:  99017
Records of all_addresses:  130000
Initial number of correct addresses:  88774
Initial coverage rate:  89.6553117141501
Improvement Rate: 41.429686628749884 %


## <a id='4'>4. Correcting Misspellings</a>  
** Assumption. No API key is available.**
- Approach 1:
  - Utilized the SpellChecker library to identify correctly spelled words within the 'address' column using the test.csv dataset.
  - Employed the list of correctly spelled words as a dictionary to correct misspelled words in the test.csv dataset.
  - Utilized the difflib library to find the closest matches for each misspelled word.
- Approach 2:
  - Utilized the Google search engine to find best-matched street names similar to the given address.
  - This method sends a search query to the Google search engine for the target address and parses the search results using BeautifulSoup.



## Approach 1

In [24]:
def extract_correct_spelled_words(addresses):
    # Initialize the SpellChecker
    spell = SpellChecker()

    # Create an empty list to store the correctly spelled words
    correct_spelled_words_list = []

    # Iterate through each address
    for address in addresses:
        words = address.split()
        correct_spelled_words = []
        for word in words:
            # Skip words that start with a digit or contain digits
            if word.isdigit() or any(char.isdigit() for char in word):
                continue

            # Check if the word is correctly spelled
#             if len(word) > 3 and len(spell.unknown([word])) == 0:
            correct_spelled_words.append(word)

        correct_spelled_words_list.append(correct_spelled_words)

    # Flatten the list and remove empty elements
    flat_list = [item for sublist in correct_spelled_words_list if sublist for item in sublist]

    # Capitalize the first letter and make the rest lowercase
    title_case_list = [word.title() for word in flat_list]

    # Remove duplicates while maintaining the order
    unique_list = []
    for item in title_case_list:
        if item not in unique_list:
            unique_list.append(item)

    return unique_list

In [25]:
selected_columns_without_nan = selected_columns[selected_columns.notna().all(axis=1)]
selected_columns_without_nan

,uid,address,city,state,zip
0,1,941 thorpe st,rock springs,wy,82901
1,2,2809 harris dr,antioch,ca,94509
2,3,1441 eastlake ave,los angeles,ca,90033
3,5,1400 lachman ln,,ca,90272
4,7,1966 clinton ave,calexico,ca,92231
...,...,...,...,...,...
90752,99996,2020 s moreno dr,apache junction,az,85120
90753,99997,10641 missouri ave,los angeles,ca,90025
90754,99998,27905 tyler ln,santa clarita,ca,91387
90755,99999,6259 w 55th st,chicago,,60638


In [26]:
# Extract and print the correctly spelled words
correct_spelled_words = extract_correct_spelled_words(selected_columns_without_nan['address'])
print(correct_spelled_words)


['Thorpe', 'St', 'Harris', 'Dr', 'Eastlake', 'Ave', 'Lachman', 'Ln', 'Clinton', 'Old', 'Goodrich', 'Rd', 'Oak', 'Creek', 'Valencia', 'Way', 'N', 'Poinsettia', 'Bryce', 'Parthenia', 'La', 'Jolla', 'Scenic', 'Hillcroft', 'Dexter', 'Leavesley', 'Pl', 'Park', 'Lento', 'Paso', 'Los', 'Cerritos', 'Rancho', 'Diegueno', 'Pacific', 'Coast', 'Hwy', 'Riverside', 'Springfield', 'Reevesbury', 'E', 'Pico', 'Hawkeye', 'Manana', 'Meadowbrook', 'Luther', 'F', 'Polk', 'Ranch', 'Hawks', 'Bill', 'Brydges', 'Ct', 'Estrella', 'Fire', 'Opal', 'Quartz', 'Hill', 'American', 'River', 'Canyon', 'Pollard', 'Via', 'Abrigada', 'Soft', 'Kenwood', 'Hamilton', 'Osler', 'Roscoe', 'Blvd', 'Arrowhead', 'Summer', 'Haven', 'Cir', 'Hile', 'Heights', 'Obregon', 'Juanita', 'Partridge', 'Lacam', 'Dena', 'Rivera', 'Avenida', 'Classica', 'Cambridge', 'Terrace', 'Paseo', 'Olivos', 'Palm', 'Tree', 'Ladrillo', 'Aisle', 'Bradley', 'Lincoln', 'Amberwood', 'Speedboat', 'Andirons', 'Luella', 'Trocha', 'De', 'Penni', 'Feather', 'Santa',

In [27]:
# Merge the two DataFrames on 'uid' to compare 'address' values
merged_df = df.merge(all_addresses_df, on='uid', how='inner', suffixes=('_df', '_ref'))
# Find rows with the same 'uid' but different 'address'
different_address_rows = merged_df[merged_df['address_df'] != merged_df['address_ref']]

# Print the resulting rows
print(different_address_rows.index)
rows_without_nan_index = different_address_rows.index


Int64Index([    3,     5,     7,    11,    15,    17,    23,    28,    32,
               35,
            ...
            98785, 98809, 98822, 98846, 98858, 98882, 98918, 98930, 98966,
            98990],
           dtype='int64', length=6877)


In [28]:
# only incorrect rows in df
test_df_mismatch = df.iloc[rows_without_nan_index]
all_addresses_df_mismatch = all_addresses_df.iloc[test_df_mismatch['uid']-1]

# Function to check if a string starts with a number
def starts_with_number(string):
    return string[0].isdigit()

# Extract numbers from addresses and create a DataFrame with numbers and corresponding indices
test_numbers_df = test_df_mismatch[test_df_mismatch['address'].apply(starts_with_number)].copy()
test_numbers_df['number'] = test_numbers_df['address'].str.extract(r'(\d+)')[0]

address_numbers_df = all_addresses_df_mismatch[all_addresses_df_mismatch['address'].apply(starts_with_number)].copy()
address_numbers_df['number'] = address_numbers_df['address'].str.extract(r'(\d+)')[0]

equal_number_idx = []
for row in range(len(test_numbers_df)):
    if int(test_numbers_df['number'].iloc[row]) == int(address_numbers_df['number'].iloc[row]):
        equal_number_idx.append(row)
matching_rows = test_numbers_df.iloc[equal_number_idx]
matching_rows


,uid,address,city,state,zip,number
3,4,7 ucayptus,newport beach,ca,92657.0,7
5,6,725 ounain view st,altadena,ca,91001.0,725
11,12,5571 moringside dr,clayton,ca,94517.0,5571
17,18,255 araise flat ln,south lake tahoe,ca,96150.0,255
23,24,525 heynne dr,san dimas,ca,91773.0,525
...,...,...,...,...,...,...
98822,99078,2151 dun ct,thousand oaks,ca,91360.0,2151
98858,99114,5140 madson rd,lucerne,ca,95458.0,5140
98918,99174,7393 acon glen loop,roseville,ca,95747.0,7393
98966,99222,21 kngson rd,berkeley,ca,94707.0,21


In [29]:
missspelled_idx = matching_rows.index
missspelled_idx

Int64Index([    3,     5,    11,    17,    23,    41,    47,    53,    65,
               71,
            ...
            98638, 98664, 98676, 98701, 98785, 98822, 98858, 98918, 98966,
            98990],
           dtype='int64', length=3550)

In [62]:
# # Define a function to correct misspelled words
# def correct_spelling(address, correct_spelled_words):
#     words_in_text = address.split()
#     for idx, word in enumerate(words_in_text):
#         # Skip words that are numbers or have numbers attached
#         if word.isdigit() or any(char.isdigit() for char in word):
#             continue

#         closest_word = difflib.get_close_matches(word, correct_spelled_words, n=1, cutoff=0.8)
#         if closest_word:
#             words_in_text[idx] = closest_word[0]  # Update the word in the list
#     return ' '.join(words_in_text)

# df.loc[missspelled_idx, 'address'] = df.loc[missspelled_idx, 'address'].apply(lambda x: correct_spelling(x, correct_spelled_words))

# # Save the result of approach 1
# df.to_csv('/content/drive/MyDrive/NLP_address_correction/approach1Result.csv', index=False)

In [30]:
# Load result of Approach 1
df_file = "/content/drive/MyDrive/NLP_address_correction/approach1Result.csv"
df = pd.read_csv(df_file)
df

,uid,address,city,state,zip
0,1,941 thorpe st,rock springs,wy,82901.0
1,2,2809 harris dr,antioch,ca,94509.0
2,3,1441 eastlake ave,los angeles,ca,90033.0
3,4,7 Eucalyptus,newport beach,ca,92657.0
4,5,1400 lachman ln,los angeles,ca,90272.0
...,...,...,...,...,...
99012,99269,2336 mountain ave,la crescenta-montrose,ca,91214.0
99013,99270,7707 mohegan ct,redding,ca,96002.0
99014,99271,1593 canby rd,redding,ca,96002.0
99015,99272,1327 mount olympia cir,south lake tahoe,ca,0.0


In [31]:
# Convert all to lower()
df.loc[:, 'address'] = df['address'].str.lower()
df.loc[:, 'city'] = df['city'].str.lower()
df.loc[:, 'state'] = df['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()

# Find number of correct addresses by merging datasets on ['uid','address', 'city', 'state', 'zip']
correct_addresses_df = pd.merge(df, all_addresses_df, on=['uid','address', 'city', 'state', 'zip'], how='inner')

# Coverage rate
imp_coverage_rate = len(correct_addresses_df)/len(df)*100

# Improvement Rate
improvement_rate = ((imp_coverage_rate - initial_coverage_rate) / initial_coverage_rate) * 100
print('Records of test: ', len(df))
print('Records of all_addresses: ', len(all_addresses_df))
print('Number of correct addresses using Approach1: ', len(correct_addresses_df))
print('Coverage rate using Approach1: ', imp_coverage_rate)
print("Improvement Rate using Approach1:", improvement_rate, "%")

Records of test:  99017
Records of all_addresses:  130000
Number of correct addresses using Approach1:  89101
Coverage rate using Approach1:  89.98555803548885
Improvement Rate using Approach1: 41.95064442638883 %


In [32]:
# Merge the two DataFrames on 'uid' to compare 'address' values
merged_df_app1 = df.merge(all_addresses_df, on='uid', how='inner', suffixes=('_df', '_ref'))
# Find rows with the same 'uid' but different 'address'
different_address_rows_app1  = merged_df_app1 [merged_df_app1 ['address_df'] != merged_df_app1 ['address_ref']]

# Print the resulting rows
print(different_address_rows_app1.index)
rows_without_nan_app1_index = different_address_rows_app1.index

Int64Index([    5,     7,    15,    17,    28,    32,    35,    41,    43,
               47,
            ...
            98748, 98785, 98809, 98822, 98846, 98858, 98882, 98918, 98930,
            98966],
           dtype='int64', length=6550)


In [33]:
print('number of miss-matched rows befor applying Approach 1:')
print(len(rows_without_nan_index))
print('number of miss-matched rows after applying Approach 1:')
print(len(rows_without_nan_app1_index))

number of miss-matched rows befor applying Approach 1:
6877
number of miss-matched rows after applying Approach 1:
6550


## Approach 2

In [34]:
def validate_address(address):
    geolocator = Nominatim(user_agent="address_validation", timeout=10)
    try:
        location = geolocator.geocode(address)
        return location
    except (GeocoderTimedOut, GeocoderUnavailable):
        return None

def calculate_similarity(address, target_address):

    similarity = 0
    parts1 = address.split()
    parts2 = target_address.split()
    for i in range(min(len(parts1),len(parts2))):
        similarity += fuzz.partial_token_set_ratio(parts1[i], parts2[i])
    return similarity


def search_best_matched_street(full_address,num_results):
    target_address = re.split(r'[,|:]', full_address)[0].strip()#f"{full_address}"
    query = full_address

    # Construct the Google search URL
    google_search_url = f"https://www.google.com/search?q={query}"
    # Send a GET request to the search URL
    response = requests.get(google_search_url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
#     title_element = soup.find("div", class_="BNeawe deIvCb AP7Wnd")
#     if title_element is not None:
#         best_match = title_element.text
#         if best_match is not None and best_match != 'Images':
#             return best_match
#         else:
#             return target_address

#     else:

    # Find all the h3 elements containing search results
    h3_elements = soup.find_all("h3", attrs={'class':'zBAuLc l97dzf'})

    # Initialize variables to store the best match
    best_match = None
    best_similarity = 0
    city_state_zip = full_address[full_address.index(','):]

    for h3_element in h3_elements:
        div_element = h3_element.find('div', class_='BNeawe vvjwJb AP7Wnd')
        if div_element:
            text = div_element.text
            street = text.split(",")[0]
            similarity = calculate_similarity(street, target_address)

            if similarity > best_similarity:
                best_similarity = similarity
                best_match = text



    if best_match is not None:
        if len(best_match.split(",")[0].split()) > 1:
            best_match_final = target_address.split()[0]+' '+' '.join(best_match.split(",")[0].split()[1:])
        else:
            best_match_final = best_match.split(",")[0]

        return best_match_final
    else:
        return target_address

In [ ]:
# num_results = 5
# for row in range(1699, len(matching_rows)):#(len(matching_rows)):#(1,2):#
#     address = matching_rows['address'].iloc[row]
#     city = matching_rows['city'].iloc[row]
#     state = matching_rows['state'].iloc[row]
#     zip_code = matching_rows['zip'].iloc[row]

#     full_address = f"{address}, {city}, {state} {zip_code}"
#     print(f"Row {matching_rows.index[row]}: {full_address}")

#     validated_address = validate_address(full_address)
#     if validated_address is None:
#         matched_address = search_best_matched_street(full_address, num_results)
#         matching_rows.loc[matching_rows.index[row], 'address'] = matched_address
#         print(colored(f"Row {matching_rows.index[row]}: {matched_address}", 'red'))



In [ ]:
# # Load test dataset
# matching_rows_file = "matching_rows_modified.csv"
# matching_rows = pd.read_csv(matching_rows_file)
# matching_rows_file
# matching_rows.index[row]

In [ ]:
# # Save the result of approach 2
# matching_rows.to_csv('/content/drive/MyDrive/NLP_address_correction/approach2Result.csv', index=False)

In [35]:
# Load obtained geo data
tapproach2Result_data_file = "/content/drive/MyDrive/NLP_address_correction/approach2Result.csv"
tapproach2Result_df = pd.read_csv(tapproach2Result_data_file)

In [36]:
tapproach2Result_df

,uid,address,city,state,zip,number
0,4,7 Eucalyptus,newport beach,ca,92657.0,7
1,6,725 Mountain View St,altadena,ca,91001.0,725
2,12,5571 Morningside Dr,clayton,ca,94517.0,5571
3,18,255 Paradise Flat Ln,south lake tahoe,ca,96150.0,255
4,24,525 Cheyenne Dr,san dimas,ca,91773.0,525
...,...,...,...,...,...,...
3545,99078,2151 Dunn Ct,thousand oaks,ca,91360.0,2151
3546,99114,5140 Madison Rd,lucerne,ca,95458.0,5140
3547,99174,7393 Acorn Glen Loop,roseville,ca,95747.0,7393
3548,99222,21 Kingston Rd,berkeley,ca,94707.0,21


In [37]:
df_app2 = df
# Merge the DataFrames based on the 'uid' column
merged_df_app2 = pd.merge(df_app2, tapproach2Result_df[['uid', 'address']], on='uid', how='left')
# Fill missing 'address' values in df1 with the corresponding values from df2
df_app2['address'] = merged_df_app2['address_y'].fillna(merged_df_app2['address_x'])
df_app2


,uid,address,city,state,zip
0,1,941 thorpe st,rock springs,wy,82901.0
1,2,2809 harris dr,antioch,ca,94509.0
2,3,1441 eastlake ave,los angeles,ca,90033.0
3,4,7 Eucalyptus,newport beach,ca,92657.0
4,5,1400 lachman ln,los angeles,ca,90272.0
...,...,...,...,...,...
99012,99269,2336 mountain ave,la crescenta-montrose,ca,91214.0
99013,99270,7707 mohegan ct,redding,ca,96002.0
99014,99271,1593 canby rd,redding,ca,96002.0
99015,99272,1327 mount olympia cir,south lake tahoe,ca,0.0


In [38]:
# Convert all to lower()
df_app2.loc[:, 'address'] = df_app2['address'].str.lower()
df_app2.loc[:, 'city'] = df_app2['city'].str.lower()
df_app2.loc[:, 'state'] = df_app2['state'].str.lower()
all_addresses_df.loc[:, 'address'] = all_addresses_df['address'].str.lower()
all_addresses_df.loc[:, 'city'] = all_addresses_df['city'].str.lower()
all_addresses_df.loc[:, 'state'] = all_addresses_df['state'].str.lower()

# Find number of correct addresses by merging datasets on ['uid','address', 'city', 'state', 'zip']
correct_addresses_df = pd.merge(df_app2, all_addresses_df, on=['uid','address', 'city', 'state', 'zip'], how='inner')

# Coverage rate
imp_coverage_rate = len(correct_addresses_df)/len(df_app2)*100

# Improvement Rate
improvement_rate = ((imp_coverage_rate - initial_coverage_rate) / initial_coverage_rate) * 100
print('Records of test: ', len(df_app2))
print('Records of all_addresses: ', len(all_addresses_df))
print('Number of correct addresses using Approach1: ', len(correct_addresses_df))
print('Coverage rate using Approach1: ', imp_coverage_rate)
print("Improvement Rate using Approach1:", improvement_rate, "%")

Records of test:  99017
Records of all_addresses:  130000
Number of correct addresses using Approach1:  91420
Coverage rate using Approach1:  92.32758011250594
Improvement Rate using Approach1: 45.64514330322295 %


In [39]:
# Merge the two DataFrames on 'uid' to compare 'address' values
merged_df_app2 = df_app2.merge(all_addresses_df, on='uid', how='inner', suffixes=('_app2', '_ref'))
# Find rows with the same 'uid' but different 'address'
different_address_rows_app2  = merged_df_app2 [merged_df_app2 ['address_app2'] != merged_df_app2 ['address_ref']]

# Print the resulting rows
print(different_address_rows_app2.index)
rows_without_nan_app2_index = different_address_rows_app2.index

Int64Index([    7,    15,    28,    32,    35,    43,    51,    55,    63,
               67,
            ...
            98457, 98553, 98626, 98676, 98748, 98785, 98809, 98846, 98882,
            98930],
           dtype='int64', length=4231)


In [40]:
print('number of miss-matched rows after applying Approach 1:')
print(len(rows_without_nan_app1_index))
print('number of miss-matched rows after applying Approach 2:')
print(len(rows_without_nan_app2_index))

number of miss-matched rows after applying Approach 1:
6550
number of miss-matched rows after applying Approach 2:
4231


In [43]:
#Here you can see the miss-matched rows:
different_address_rows_app2

,uid,address_app2,city_app2,state_app2,zip_app2,address_ref,city_ref,state_ref,zip_ref
7,8,431 6th st,sacramento,ca,95820.0,4341 69th st,sacramento,ca,95820
15,16,36 w aomi ave,arcadia,ca,91007.0,365 w naomi ave,arcadia,ca,91007
28,28,1104 eatherwood terrace,san diego,ca,92131.0,11004 weatherwood terrace,san diego,ca,92131
32,32,179 nwilton pl,los angeles,ca,90028.0,1719 n wilton pl,los angeles,ca,90028
35,36,1202 gad ave,norwalk,ca,90650.0,12025 gard ave,norwalk,ca,90650
...,...,...,...,...,...,...,...,...,...
98785,99042,906 west creston street,santa maria,ca,93458.0,906 w creston st,santa maria,ca,93458
98809,99066,1523 cale naranjo,santa clarita,ca,91390.0,15230 calle naranjo,santa clarita,ca,91390
98846,99102,1464 pak ln,midway city,ca,92655.0,14642 park ln,midway city,ca,92655
98882,99138,8569 cl limon,coachella,ca,92236.0,85697 cll limon,coachella,ca,92236
